# Analise de sentimentos de tweets em português utilizando o Watson Natural Language Understanding <br>
Lembrando que você pode utilizar a API do twitter e o Watson Natural Language Understanding de forma gratuita, porém com algumas limitações.

In [1]:
import os
from dotenv import load_dotenv
import tweepy as tw
import pandas as pd

from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, SentimentOptions
import re

In [2]:
pd.set_option('display.max_colwidth', 240)

# Parte 1 - Usando a API do twitter

Carregando as informações de autenticação

In [3]:
load_dotenv()
api_key = os.getenv("CONSUMER_KEY")
api_key_secret = os.getenv("CONSUMER_KEY_SECRET")
access_token = os.getenv("ACCESS_TOKEN")
access_token_secret = os.getenv("ACCESS_TOKEN_SECRET")

Autenticando

In [4]:
auth = tw.OAuthHandler(api_key,api_key_secret)
auth.set_access_token(access_token,access_token_secret)
#api = tw.API(auth, wait_on_rate_limit=True)
api = tw.API(auth)

Procurando tweets com a palavra BBB e que não sejam retweets e buscando por tweets recentes. <br>
Trouxe apenas 10 tweets para poder utiliza-los no Watson Natural Language Understanding.

In [22]:
search_words = "BBB" + "-filter:retweets"
tweets = tw.Cursor(api.search_tweets,result_type="recent", q=search_words).items(10)

Percorrendo pelo objeto do tweetpy e salvando as informações em uma lista e depois gerando um dataframe com esses dados

In [24]:
lst_text = []
lst_data = []
for tweet in tweets:
    lst_data.append(tweet.created_at)
    lst_text.append(tweet.text)

In [25]:
df_tweets = pd.DataFrame({"Texto":lst_text, "Data":lst_data})

Aqui está os últimos tweets sobre BBB

In [26]:
df_tweets.head(20)

,Texto,Data
0,nas festas do BBB os marmanjo tudo vira adolescente kkkkkkkkkkkkk,2022-01-24 02:22:32+00:00
1,gente Lucas nem aparece nesse bbb né,2022-01-24 02:22:32+00:00
2,A Bruna consegue ser mais metida que a Jade #bbb #bbb2022,2022-01-24 02:22:32+00:00
3,Edição do bbb mostrando na cara do brasileiro o quanto Rodrigo é chato e INCOVENIENTE #bbb2022,2022-01-24 02:22:32+00:00
4,"Gente, só queria me alienar!\nBbb tá chato para caralho",2022-01-24 02:22:32+00:00
5,#BBB22 AO VIVO BBB https://t.co/EEoFCrBvTu,2022-01-24 02:22:32+00:00
6,"rapaz, bbb me parece tanto festinha de adolescente",2022-01-24 02:22:31+00:00
7,"Esse ano o campeão do bbb não sai da pipoca nem fudendo, pessoal mó sem carisma #BBB22",2022-01-24 02:22:31+00:00
8,"A narrativa do BBB é sempre a mesma ultimamente. Os brancos nojentos de classe média alta, se unem contra os negros… https://t.co/uqQHJerBhb",2022-01-24 02:22:31+00:00
9,Retiro o que eu disse no 1° dia de BBB 😕,2022-01-24 02:22:31+00:00


# Parte 2 Utilizando o Watson Natural Language Understanding

Carregandos as informações de autenticação e autenticando

In [17]:
NLU_key = os.getenv("NLU_KEY")
NLU_url = os.getenv("NLU_URL")

In [18]:
authenticator = IAMAuthenticator(NLU_key)
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2021-08-01',
    authenticator=authenticator
)
natural_language_understanding.set_service_url(NLU_url)

Função trazer o resultado da analise

In [19]:
def analise_sentimento(row):
    response = natural_language_understanding.analyze(
        text=row["Texto"],
        features=Features(sentiment=SentimentOptions(targets=['BBB'])),
        language="pt").get_result()
    row["Documento Score"] = response["sentiment"]["document"]["score"]
    row["Documento Label"] = response["sentiment"]["document"]["label"]
    row["Score"] = response["sentiment"]["targets"][0]["score"]
    row["Label"] = response["sentiment"]["targets"][0]["label"]
    return row

In [20]:
df_analise = df_tweets.apply(analise_sentimento, axis=1)

In [21]:
df_analise.head(20)

,Texto,Data,Documento Score,Documento Label,Score,Label
0,"meu deus, eu ia me irritar mt no bbb pq toda festa vem um bebado insuportavel de conversinha chata",2022-01-24 02:17:40+00:00,-0.998627,negative,-0.998627,negative
1,imagina tu tá na festa do bbb e a pessoa chega pra falar de jogo,2022-01-24 02:17:40+00:00,0.000000,neutral,0.000000,neutral
2,Comi assim é difícil tá no BBB?????,2022-01-24 02:17:40+00:00,-0.903788,negative,-0.903788,negative
3,gente esse BBB ta tão 💤,2022-01-24 02:17:39+00:00,-0.488388,negative,-0.488388,negative
4,Mano o Douglas tá mt paciente isso sim. Se fosse eu já tinha mandando tomar no cú e pronto\n\n#BBB\n#BBB2022\n#BBB22,2022-01-24 02:17:39+00:00,-0.752188,negative,0.000000,neutral
5,@thaisbraz Thais e BBB 22,2022-01-24 02:17:39+00:00,0.532432,positive,0.532432,positive
6,assistir o bbb ao vivo no globo play é pedir pra irritar,2022-01-24 02:17:39+00:00,-0.961629,negative,-0.961629,negative
7,eu vejo td que passa na edição ao vivo tô me sentindo mt viciada em bbb,2022-01-24 02:17:39+00:00,0.388700,positive,0.388700,positive
8,a menina falando c o Douglas agr no BBB KAKSKAKSKAKDKAKKS,2022-01-24 02:17:39+00:00,0.000000,neutral,0.000000,neutral
9,Teve taylor no bbb SIM\n🎶 Like like tuntuntun 🎶,2022-01-24 02:17:39+00:00,0.000000,neutral,0.000000,neutral


Para uma melhor analise dos tweets é recomendado uma limpeza no texto.